<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/CA2/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle/
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ ;
!chmod 600 ~/.kaggle/kaggle.json ; 
!kaggle datasets download -d ankurzing/sentiment-analysis-for-financial-news ;
!unzip /content/sentiment-analysis-for-financial-news.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle/’: File exists
sentiment-analysis-for-financial-news.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/sentiment-analysis-for-financial-news.zip
replace FinancialPhraseBank/License.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:
dataset = pd.read_csv("./all-data.csv", encoding='iso-8859-1', header=None)
dataset.columns = ["sentiment","phrase"]

In [4]:
dataset["sentiment"].value_counts()

neutral     2879
positive    1363
negative     604
Name: sentiment, dtype: int64

In [59]:


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

In [60]:
encoder = LabelEncoder()
dataset["label"] = encoder.fit_transform(dataset["sentiment"])
dataset

,sentiment,phrase,label
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,1
2,negative,The international electronic industry company ...,0
3,positive,With the new production plant the company woul...,2
4,positive,According to the company 's updated strategy f...,2
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,0
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,negative,Net sales of the Paper segment decreased to EU...,0


In [61]:
dataset["phrase"] = dataset["phrase"].apply(preprocess)

In [9]:
dataset["phrase"]

0       [according, gran, company, plan, move, product...
1       [technopolis, plan, develop, stage, area, le, ...
2       [international, electronic, industry, company,...
3       [new, production, plant, company, would, incre...
4       [according, company, 's, updated, strategy, ye...
                              ...                        
4841    [london, marketwatch, --, share, price, ended,...
4842    [rinkuskiai, 's, beer, sale, fell, 6.5, per, c...
4843    [operating, profit, fell, eur, 35.4, mn, eur, ...
4844    [net, sale, paper, segment, decreased, eur, 22...
4845    [sale, finland, decreased, 10.5, january, sale...
Name: phrase, Length: 4846, dtype: object

In [10]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove.6B


--2023-05-06 14:37:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-06 14:37:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-06 14:37:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

In [11]:

embeddings_index = {}
with open('glove.6B/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embeddings = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embeddings


In [64]:

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(dataset['phrase'])

phrases = tokenizer.texts_to_sequences(dataset['phrase'])
labels = dataset['label']


maxlen = 100
X = pad_sequences(phrases, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

def fit_embedding(X):
    X_embedded = list()
    for phrase in X:
        phrase_embedded = np.zeros((maxlen, embedding_dim))
        for indx, word in enumerate(phrase):
          phrase_embedded[indx] = embedding_matrix[word]
        X_embedded.append(np.mean(phrase_embedded, axis=0))
            
    return np.array(X_embedded)

X = fit_embedding(X)


X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, stratify=labels)


clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_test, y_pred, target_names=target_names))


              precision    recall  f1-score   support

    Negative       0.70      0.11      0.20        61
     Neutral       0.68      0.93      0.79       288
    Positive       0.49      0.31      0.38       136

    accuracy                           0.65       485
   macro avg       0.63      0.45      0.45       485
weighted avg       0.63      0.65      0.60       485

